In [1]:
import pennylane as qml
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from math import log, ceil
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [29]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev, interface="torch")
def circuit(inputs, theta):
    qml.AmplitudeEmbedding(inputs, normalize=True, wires=[0, 1])
    qml.StronglyEntanglingLayers(theta, wires=[0, 1])
    return qml.expval(qml.PauliZ(0))

class QNNModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.q_layer = qml.qnn.TorchLayer(circuit, {
            "theta": (2,2,3,)
        })
    
    def forward(self, x):
        x = self.q_layer(x)
        return x

In [30]:
model = QNNModel()


In [26]:
data_x = torch.tensor([[1.0, 0.0, 0.0, 0.0]]*50 + [[0.0, 1.0, 0.0, 0.0]]*50)
data_y = torch.tensor([1.0]*50 + [-1.0]*50)

In [27]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [31]:
model.train()
for epoch in tqdm(range(100)):
    optimizer.zero_grad()
    loss = F.mse_loss(model(data_x), data_y)
    loss.backward()
    optimizer.step()

100%|██████████| 100/100 [00:02<00:00, 36.13it/s]


In [34]:
model(torch.tensor([1.0, 0.0, 0.0, 0.0]))

tensor(-0.8410, grad_fn=<ToCopyBackward0>)